<a href="https://colab.research.google.com/github/Surya23-Git/Web_Scrapping/blob/main/WebScrapping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
from bs4 import BeautifulSoup

In [65]:
for i in range(1,5):
  url = f"https://www.fifaindex.com/?page={i}"
  print(url)

https://www.fifaindex.com/?page=1
https://www.fifaindex.com/?page=2
https://www.fifaindex.com/?page=3
https://www.fifaindex.com/?page=4


In [140]:
df = pd.DataFrame()
# all_data = []
# Image_link = []
# Player_Name = []
# Player_Age = []
# Player_OVR = []
# Player_POT = []
# Player_Team = []
# Player_Position = []
# Player_Country = []
all_data, Image_link, Player_Name, Player_Age, Player_OVR, Player_POT, Player_Team, Player_Position, Player_Country, Pos_list= ([] for i in range(10))
for i in range(1,5):
  req = requests.get(f"https://www.fifaindex.com/?page={i}")
  soup = BeautifulSoup(req.content)
  soup.prettify()
  arr = [j for j in soup.findAll('tr')]
  del arr[0]
  
  for k in range(0,len(arr)):
    Image_link.append(re.sub('.*="|".*', '', str(arr[k].findAll('a', {'class':'link-player'})[0])))
    Player_Name.append(re.sub('.*title="| FIFA.*', '', str(arr[k].findAll('a', {'class':'link-player'})[0])))
    Player_Country.append(re.sub('.*title="|".*', '', str(arr[k].findAll('a', {'class':'link-nation'})[0])))
    Player_Age.append(re.sub('.*">|<.*', '', str(re.search('<td data-title="Age">(\d+)</td>',str(arr[k])))))
    Player_OVR.append(re.sub('.*r3">|.*r1">|.*r2">|<.*', '', str(arr[k])))
    Player_POT.append(re.sub('.*r3">|.*r1">|.*r2">|<.*', '', str(arr[k])))
    Player_Team.append(re.sub('.*title="| FIFA.*', '', str(arr[k].findAll('a', {'class':'link-team'})[0])))
    Pos_list = []
    for m in range(len(arr[k].findAll('a', {'class':'link-position'}))):
      Pos_list.append(re.sub('.*title="|".*', '', str(arr[k].findAll('a', {'class':'link-position'})[m])))
    Player_Position.append(Pos_list)
    #Player_Position.append(re.sub('.*title="|".*', '', str(arr[k].findAll('a', {'class':'link-position'})[0])))
  data = pd.DataFrame({
        'Name': Player_Name,
        'Age': Player_Age,
        'Country': Player_Country,
        'OVR': Player_OVR,
        'POT': Player_POT,
        'Team': Player_Team,
        'Position': Player_Position,
        'Image_link': Image_link
        
    })
  
all_data.append(data)

In [142]:
Players_df = df.append(all_data, ignore_index=True)
Players_df["Name"] = Players_df["Name"].str.strip()
Players_df["Team"] = Players_df["Team"].str.strip()
#Players_df["Position"] = Players_df["Position"].str.strip()
Players_df["Image_link"] = Players_df["Image_link"].str.strip()
Players_df

,Name,Age,Country,OVR,POT,Team,Position,Image_link
0,Mykhailo Mudryk,22,Ukraine,84,84,Shakhtar Donetsk,"[LM, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
1,Kaoru Mitoma,25,Japan,77,77,Brighton &amp; Hove Albion,"[LM, LW, LWB]",https://fifastatic.fifaindex.com/FIFA23/player...
2,Enzo Fernández,21,Argentina,87,87,Benfica,"[CM, CDM]",https://fifastatic.fifaindex.com/FIFA23/player...
3,Jayden Oosterwolde,21,Netherlands,79,79,Parma,"[LB, LM, LWB]",https://fifastatic.fifaindex.com/FIFA23/player...
4,Khvicha Kvaratskhelia,21,Georgia,84,84,Napoli,"[LW, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
...,...,...,...,...,...,...,...,...
95,Cody Gakpo,23,Netherlands,87,87,PSV,[LW],https://fifastatic.fifaindex.com/FIFA23/player...
96,Gonçalo Inácio,21,Portugal,88,88,Sporting CP,[CB],https://fifastatic.fifaindex.com/FIFA23/player...
97,Anthony Elanga,20,Sweden,85,85,Manchester United,"[RW, LW, RM]",https://fifastatic.fifaindex.com/FIFA23/player...
98,Wilfried Gnonto,19,Italy,85,85,FC Zürich,"[ST, LM]",https://fifastatic.fifaindex.com/FIFA23/player...


In [143]:
Players_df.Age = Players_df.Age.astype(int)
Players_df.OVR = Players_df.OVR.astype(int)
Players_df.POT = Players_df.POT.astype(int)

In [144]:
Players_df.to_csv("Players_Refined.csv")

In [145]:
Players_df.isna().sum()

Name          0
Age           0
Country       0
OVR           0
POT           0
Team          0
Position      0
Image_link    0
dtype: int64

In [146]:
Players_df.columns

Index(['Name', 'Age', 'Country', 'OVR', 'POT', 'Team', 'Position',
       'Image_link'],
      dtype='object')

In [147]:
Players_df[["OVR", "POT", "Age"]].describe()

,OVR,POT,Age
count,100.000000,100.000000,100.000000
mean,84.690000,84.690000,23.440000
std,4.466927,4.466927,3.827914
min,74.000000,74.000000,19.000000
25%,82.000000,82.000000,21.000000
50%,85.000000,85.000000,22.000000
75%,88.000000,88.000000,25.000000
max,95.000000,95.000000,37.000000


In [148]:
Players_df[Players_df["Name"] == "Erling Haaland"]

,Name,Age,Country,OVR,POT,Team,Position,Image_link
6,Erling Haaland,22,Norway,94,94,Manchester City,[ST],https://fifastatic.fifaindex.com/FIFA23/player...


In [149]:
Players_df.sort_values(by=["Age", "OVR"])

,Name,Age,Country,OVR,POT,Team,Position,Image_link
12,Emre Demir,19,Turkey,79,79,FC Barcelona,"[CAM, RW, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
17,Malo Gusto,19,France,85,85,Olympique Lyonnais,[RB],https://fifastatic.fifaindex.com/FIFA23/player...
98,Wilfried Gnonto,19,Italy,85,85,FC Zürich,"[ST, LM]",https://fifastatic.fifaindex.com/FIFA23/player...
44,Xavi Simons,19,Netherlands,86,86,PSV,"[CAM, CM, RW]",https://fifastatic.fifaindex.com/FIFA23/player...
58,Carney Chukwuemeka,19,England,86,86,Chelsea,"[CM, CAM, CF]",https://fifastatic.fifaindex.com/FIFA23/player...
...,...,...,...,...,...,...,...,...
76,Danilo,31,Brazil,80,80,Juventus,[RB],https://fifastatic.fifaindex.com/FIFA23/player...
42,Jorginho,31,Italy,85,85,Chelsea,[CM],https://fifastatic.fifaindex.com/FIFA23/player...
32,Salomón Rondón,33,Venezuela,74,74,Everton,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
75,Luis Suárez,35,Uruguay,84,84,Nacional de Montevideo,[ST],https://fifastatic.fifaindex.com/FIFA23/player...


In [150]:
Players_df.sort_values(by=["Age", "OVR"], ascending = [True, True]).head()

,Name,Age,Country,OVR,POT,Team,Position,Image_link
12,Emre Demir,19,Turkey,79,79,FC Barcelona,"[CAM, RW, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
17,Malo Gusto,19,France,85,85,Olympique Lyonnais,[RB],https://fifastatic.fifaindex.com/FIFA23/player...
98,Wilfried Gnonto,19,Italy,85,85,FC Zürich,"[ST, LM]",https://fifastatic.fifaindex.com/FIFA23/player...
44,Xavi Simons,19,Netherlands,86,86,PSV,"[CAM, CM, RW]",https://fifastatic.fifaindex.com/FIFA23/player...
58,Carney Chukwuemeka,19,England,86,86,Chelsea,"[CM, CAM, CF]",https://fifastatic.fifaindex.com/FIFA23/player...


In [151]:
Players_df.sort_values(by=["Age", "OVR"], ascending = [False, True]).head()

,Name,Age,Country,OVR,POT,Team,Position,Image_link
7,Cristiano Ronaldo,37,Portugal,90,90,Manchester United,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
75,Luis Suárez,35,Uruguay,84,84,Nacional de Montevideo,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
32,Salomón Rondón,33,Venezuela,74,74,Everton,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
24,Fabio Borini,31,Italy,75,75,Fatih Karagümrük S.K.,"[LW, ST, RW]",https://fifastatic.fifaindex.com/FIFA23/player...
76,Danilo,31,Brazil,80,80,Juventus,[RB],https://fifastatic.fifaindex.com/FIFA23/player...


In [152]:
Players_df.sort_values(by=["OVR", "Age"], ascending = [False, True]).head()

,Name,Age,Country,OVR,POT,Team,Position,Image_link
11,Kylian Mbappé,24,France,95,95,Paris Saint-Germain,"[ST, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
6,Erling Haaland,22,Norway,94,94,Manchester City,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
23,Vinícius Jr.,22,Brazil,92,92,Real Madrid,[LW],https://fifastatic.fifaindex.com/FIFA23/player...
83,Phil Foden,22,England,92,92,Manchester City,"[LW, CF, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
38,Florian Wirtz,19,Germany,91,91,Bayer 04 Leverkusen,"[CAM, CM]",https://fifastatic.fifaindex.com/FIFA23/player...


In [153]:
# 10 Youngest Players in the dataset
Young = Players_df.sort_values(by=["Age"], ascending = [True]).head(10)["Name"]
Young

49         Jamal Musiala
28        Harvey Elliott
44           Xavi Simons
98       Wilfried Gnonto
58    Carney Chukwuemeka
17            Malo Gusto
38         Florian Wirtz
12            Emre Demir
37          Rayan Cherki
9         Joško Gvardiol
Name: Name, dtype: object

In [154]:
Players_df.sort_values(by=["Age"], ascending = [True]).head(10)

,Name,Age,Country,OVR,POT,Team,Position,Image_link
49,Jamal Musiala,19,Germany,90,90,Bayern München,"[CM, LM, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
28,Harvey Elliott,19,England,87,87,Liverpool,"[CAM, CM, RW]",https://fifastatic.fifaindex.com/FIFA23/player...
44,Xavi Simons,19,Netherlands,86,86,PSV,"[CAM, CM, RW]",https://fifastatic.fifaindex.com/FIFA23/player...
98,Wilfried Gnonto,19,Italy,85,85,FC Zürich,"[ST, LM]",https://fifastatic.fifaindex.com/FIFA23/player...
58,Carney Chukwuemeka,19,England,86,86,Chelsea,"[CM, CAM, CF]",https://fifastatic.fifaindex.com/FIFA23/player...
17,Malo Gusto,19,France,85,85,Olympique Lyonnais,[RB],https://fifastatic.fifaindex.com/FIFA23/player...
38,Florian Wirtz,19,Germany,91,91,Bayer 04 Leverkusen,"[CAM, CM]",https://fifastatic.fifaindex.com/FIFA23/player...
12,Emre Demir,19,Turkey,79,79,FC Barcelona,"[CAM, RW, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
37,Rayan Cherki,19,France,88,88,Olympique Lyonnais,"[LW, ST, RW]",https://fifastatic.fifaindex.com/FIFA23/player...
9,Joško Gvardiol,20,Croatia,89,89,RB Leipzig,"[CB, LB]",https://fifastatic.fifaindex.com/FIFA23/player...


In [155]:
# Top 5 Younger Players with highest OVR rating
OVR_Young = Players_df.sort_values(by=["OVR", "Age"], ascending = [False, True]).head()
OVR_Young

,Name,Age,Country,OVR,POT,Team,Position,Image_link
11,Kylian Mbappé,24,France,95,95,Paris Saint-Germain,"[ST, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
6,Erling Haaland,22,Norway,94,94,Manchester City,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
23,Vinícius Jr.,22,Brazil,92,92,Real Madrid,[LW],https://fifastatic.fifaindex.com/FIFA23/player...
83,Phil Foden,22,England,92,92,Manchester City,"[LW, CF, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
38,Florian Wirtz,19,Germany,91,91,Bayer 04 Leverkusen,"[CAM, CM]",https://fifastatic.fifaindex.com/FIFA23/player...


In [156]:
# Top 5 Younger Players with highest POT rating
POT_Young = Players_df.sort_values(by=["POT","Age"], ascending = [False, True]).head()
POT_Young

,Name,Age,Country,OVR,POT,Team,Position,Image_link
11,Kylian Mbappé,24,France,95,95,Paris Saint-Germain,"[ST, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
6,Erling Haaland,22,Norway,94,94,Manchester City,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
23,Vinícius Jr.,22,Brazil,92,92,Real Madrid,[LW],https://fifastatic.fifaindex.com/FIFA23/player...
83,Phil Foden,22,England,92,92,Manchester City,"[LW, CF, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
38,Florian Wirtz,19,Germany,91,91,Bayer 04 Leverkusen,"[CAM, CM]",https://fifastatic.fifaindex.com/FIFA23/player...


In [157]:
OVR_TOP = Players_df.sort_values(by=["OVR"], ascending = False).head()
OVR_TOP

,Name,Age,Country,OVR,POT,Team,Position,Image_link
11,Kylian Mbappé,24,France,95,95,Paris Saint-Germain,"[ST, LW]",https://fifastatic.fifaindex.com/FIFA23/player...
6,Erling Haaland,22,Norway,94,94,Manchester City,[ST],https://fifastatic.fifaindex.com/FIFA23/player...
23,Vinícius Jr.,22,Brazil,92,92,Real Madrid,[LW],https://fifastatic.fifaindex.com/FIFA23/player...
83,Phil Foden,22,England,92,92,Manchester City,"[LW, CF, CAM]",https://fifastatic.fifaindex.com/FIFA23/player...
84,Dušan Vlahović,22,Serbia,91,91,Juventus,[ST],https://fifastatic.fifaindex.com/FIFA23/player...


In [158]:
# Reading the saved csv file
df = pd.read_csv("/content/Players_Refined.csv")

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   Name        100 non-null    object
 2   Age         100 non-null    int64 
 3   Country     100 non-null    object
 4   OVR         100 non-null    int64 
 5   POT         100 non-null    int64 
 6   Team        100 non-null    object
 7   Position    100 non-null    object
 8   Image_link  100 non-null    object
dtypes: int64(4), object(5)
memory usage: 7.2+ KB


In [160]:
df.drop(['Unnamed: 0'], axis=1, inplace = True)

In [161]:
df

,Name,Age,Country,OVR,POT,Team,Position,Image_link
0,Mykhailo Mudryk,22,Ukraine,84,84,Shakhtar Donetsk,"['LM', 'CAM']",https://fifastatic.fifaindex.com/FIFA23/player...
1,Kaoru Mitoma,25,Japan,77,77,Brighton &amp; Hove Albion,"['LM', 'LW', 'LWB']",https://fifastatic.fifaindex.com/FIFA23/player...
2,Enzo Fernández,21,Argentina,87,87,Benfica,"['CM', 'CDM']",https://fifastatic.fifaindex.com/FIFA23/player...
3,Jayden Oosterwolde,21,Netherlands,79,79,Parma,"['LB', 'LM', 'LWB']",https://fifastatic.fifaindex.com/FIFA23/player...
4,Khvicha Kvaratskhelia,21,Georgia,84,84,Napoli,"['LW', 'CAM']",https://fifastatic.fifaindex.com/FIFA23/player...
...,...,...,...,...,...,...,...,...
95,Cody Gakpo,23,Netherlands,87,87,PSV,['LW'],https://fifastatic.fifaindex.com/FIFA23/player...
96,Gonçalo Inácio,21,Portugal,88,88,Sporting CP,['CB'],https://fifastatic.fifaindex.com/FIFA23/player...
97,Anthony Elanga,20,Sweden,85,85,Manchester United,"['RW', 'LW', 'RM']",https://fifastatic.fifaindex.com/FIFA23/player...
98,Wilfried Gnonto,19,Italy,85,85,FC Zürich,"['ST', 'LM']",https://fifastatic.fifaindex.com/FIFA23/player...


In [162]:
df[df["Name"] == "Anthony Gordon"]

,Name,Age,Country,OVR,POT,Team,Position,Image_link
18,Anthony Gordon,21,England,84,84,Everton,"['LM', 'RM']",https://fifastatic.fifaindex.com/FIFA23/player...


In [164]:
df[df["Position"].str.contains("ST")]

,Name,Age,Country,OVR,POT,Team,Position,Image_link
6,Erling Haaland,22,Norway,94,94,Manchester City,['ST'],https://fifastatic.fifaindex.com/FIFA23/player...
7,Cristiano Ronaldo,37,Portugal,90,90,Manchester United,['ST'],https://fifastatic.fifaindex.com/FIFA23/player...
11,Kylian Mbappé,24,France,95,95,Paris Saint-Germain,"['ST', 'LW']",https://fifastatic.fifaindex.com/FIFA23/player...
13,Victor Osimhen,24,Nigeria,89,89,Napoli,['ST'],https://fifastatic.fifaindex.com/FIFA23/player...
14,Julián Álvarez,22,Argentina,86,86,Manchester City,"['ST', 'RW', 'LW']",https://fifastatic.fifaindex.com/FIFA23/player...
15,Karim Adeyemi,20,Germany,87,87,Borussia Dortmund,['ST'],https://fifastatic.fifaindex.com/FIFA23/player...
16,Sardar Azmoun,28,Iran,81,81,Bayer 04 Leverkusen,"['ST', 'CAM']",https://fifastatic.fifaindex.com/FIFA23/player...
19,Alexis Vega,25,Mexico,83,83,NG - FA,"['LW', 'ST', 'LM']",https://fifastatic.fifaindex.com/FIFA23/player...
22,Luka Jović,25,Serbia,84,84,Fiorentina,['ST'],https://fifastatic.fifaindex.com/FIFA23/player...
24,Fabio Borini,31,Italy,75,75,Fatih Karagümrük S.K.,"['LW', 'ST', 'RW']",https://fifastatic.fifaindex.com/FIFA23/player...
